# Laboratorium nr 10

In [ ]:
# Importy niezbędnych paczek
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Ensemble i Soft Voting

## Czym są metody ensemble?

Metody ensemble (ang. ensemble methods) to techniki w uczeniu maszynowym, które łączą przewidywania wielu modeli (tzw. modeli bazowych) w celu uzyskania lepszej dokładności i stabilności predykcji niż pojedynczy model. Ideą jest wykorzystanie różnorodności modeli, aby zredukować błędy wynikające z nadmiernego dopasowania (overfitting), wariancji lub obciążenia (bias).

### Główne typy metod ensemble:
1. **Bagging** (Bootstrap Aggregating): Modele trenuje się na różnych podzbiorach danych (losowo wybieranych z powtórzeniami), a wyniki są agregowane, np. przez średnią (dla regresji) lub głosowanie większościowe (dla klasyfikacji). Przykład: Random Forest.
2. **Boosting**: Modele trenuje się sekwencyjnie, gdzie każdy kolejny model skupia się na poprawianiu błędów poprzednich. Przykład: Gradient Boosting, AdaBoost.
3. **Stacking**: Wyniki wielu modeli są łączone za pomocą meta-modelu, który uczy się, jak najlepiej ważyć przewidywania modeli bazowych.
4. **Voting**: Przewidywania wielu modeli są łączone przez głosowanie (dla klasyfikacji) lub średnią (dla regresji). Wyróżniamy **hard voting** i **soft voting**.

---

## Soft Voting - Szczegóły i Matematyka

### Czym jest soft voting?

Soft voting to technika ensemble dla problemów klasyfikacji, w której przewidywania modeli bazowych są łączone na podstawie **prawdopodobieństw przynależności do klas**. Każdy model bazowy zwraca prawdopodobieństwa dla każdej klasy, a końcowa predykcja jest obliczana jako średnia ważona tych prawdopodobieństw, po czym wybierana jest klasa o najwyższym średnim prawdopodobieństwie.

W przeciwieństwie do **hard voting**, gdzie każdy model oddaje jeden "głos" na klasę (wynik jest liczony jako większość głosów), soft voting uwzględnia pewność modelu wyrażoną w prawdopodobieństwach, co zazwyczaj prowadzi do lepszych wyników.

### Formalny opis matematyczny

Załóżmy, że mamy:
- $ M $ modeli bazowych (np. drzewa decyzyjne, SVM, sieci neuronowe).
- $ K $ klas, które przewidujemy (np. $ K = 3 $ dla klas: "kot", "pies", "ptak").
- Dla każdego modelu $ m $ (gdzie $ m = 1, 2, \dots, M $) i dla każdej próbki $ x $, model zwraca wektor prawdopodobieństw przynależności do klas:
  $$
  P_m(x) = [p_{m,1}(x), p_{m,2}(x), \dots, p_{m,K}(x)],
  $$
  gdzie $ p_{m,k}(x) $ to prawdopodobieństwo, że próbka $ x $ należy do klasy $ k $ według modelu $ m $, oraz $ \sum_{k=1}^K p_{m,k}(x) = 1 $.

#### Krok 1: Obliczenie średnich prawdopodobieństw
Dla każdej klasy $ k $, obliczamy średnie prawdopodobieństwo na podstawie wszystkich modeli:
$$
\bar{p}_k(x) = \frac{1}{M} \sum_{m=1}^M p_{m,k}(x).
$$
Jeśli modele mają różne wagi $ w_m $ (gdzie $ \sum_{m=1}^M w_m = 1 $), to:
$$
\bar{p}_k(x) = \sum_{m=1}^M w_m \cdot p_{m,k}(x).
$$

#### Krok 2: Wybór klasy
Końcowa predykcja to klasa $ k^* $, która ma najwyższe średnie prawdopodobieństwo:
$$
k^* = \arg\max_{k \in \{1, 2, \dots, K\}} \bar{p}_k(x).
$$

### Przykład
Załóżmy, że mamy 3 modele ($ M = 3 $) i 2 klasy ($ K = 2 $, np. "pozytywna" i "negatywna"). Dla próbki $ x $, modele zwracają następujące prawdopodobieństwa:

- Model 1: $ P_1(x) = [0.9, 0.1] $ (90% na klasę pozytywną, 10% na negatywną).
- Model 2: $ P_2(x) = [0.7, 0.3] $.
- Model 3: $ P_3(x) = [0.6, 0.4] $.

Obliczamy średnie prawdopodobieństwa dla każdej klasy:
$$
\bar{p}_1(x) = \frac{0.9 + 0.7 + 0.6}{3} = 0.733,
$$
$$
\bar{p}_2(x) = \frac{0.1 + 0.3 + 0.4}{3} = 0.267.
$$
Klasa o najwyższym prawdopodobieństwie to klasa 1 (pozytywna), więc $ k^* = 1 $.

---

## Zalety i wady soft voting

### Zalety:
- Uwzględnia pewność modeli, co prowadzi do bardziej wyważonych decyzji.
- Często daje lepsze wyniki niż hard voting, szczególnie gdy modele są dobrze skalibrowane.
- Elastyczność w stosowaniu wag dla różnych modeli.

### Wady:
- Wymaga, aby modele zwracały dobrze skalibrowane prawdopodobieństwa.
- Większa złożoność obliczeniowa w porównaniu do hard voting.
- Może być mniej skuteczne, jeśli modele są bardzo różne pod względem jakości predykcji.

---


### Zadanie nr 1 (5 pkt)
Zaimplementuj klasyfikator ensemble z soft voting w Pythonie. Porównaj wyniki z wbudowanym modelem `VotingClassifier`.

1. Wygeneruj syntetyczny zbiór danych binarnej klasyfikacji (np. za pomocą `make_classification` z scikit-learn).
2. Zbuduj trzy różne modele bazowe (np. Logistic Regression, Decision Tree, SVM).
3. Zaimplementuj soft voting ręcznie, obliczając średnie prawdopodobieństwa dla każdej klasy.
4. Porównaj wyniki swojej implementacji z modelem `VotingClassifier` z scikit-learn.
5. Oceń dokładność (accuracy) obu podejść na zbiorze testowym.

# Lasy Losowe

## Czym są lasy losowe?

Lasy losowe (ang. Random Forests) to metoda ensemble oparta na technice **bagging** (Bootstrap Aggregating), która łączy wiele drzew decyzyjnych w celu poprawy dokładności i stabilności predykcji. Została zaproponowana przez Leo Breimana w 2001 roku i jest szeroko stosowana zarówno w zadaniach klasyfikacji, jak i regresji.

### Kluczowe cechy lasów losowych:
1. **Wiele drzew decyzyjnych**: Las losowy składa się z $ T $ drzew decyzyjnych, gdzie każde drzewo jest trenowane na losowym podzbiorze danych.
2. **Bootstrap**: Każde drzewo jest trenowane na losowym podzbiorze danych treningowych, wybranym z powtórzeniami (tzw. bootstrap sample). Zazwyczaj wielkość podzbioru jest równa wielkości oryginalnego zbioru danych.
3. **Losowy wybór cech**: W każdym węźle drzewa decyzyjnego wybierana jest losowa podgrupa cech (atrybutów), spośród których wybierana jest najlepsza cecha do podziału. To zmniejsza korelację między drzewami i zwiększa różnorodność.
4. **Agregacja wyników**:
   - Dla **klasyfikacji**: Wyniki drzew są łączone przez głosowanie większościowe (ang. majority voting).
   - Dla **regresji**: Wyniki drzew są uśredniane.

### Formalny opis matematyczny

Załóżmy, że mamy zbiór danych treningowych $ D = \{(x_1, y_1), (x_2, y_2), \dots, (x_N, y_N)\} $, gdzie $ x_i $ to wektor cech, a $ y_i $ to etykieta (dla klasyfikacji) lub wartość ciągła (dla regresji).

#### Krok 1: Tworzenie podzbiorów danych
Dla każdego drzewa $ t = 1, 2, \dots, T $:
- Losujemy z powtórzeniami podzbiór danych $ D_t $ o rozmiarze $ N $ (bootstrap sample).
- Losowy podzbiór cech (np. $ \sqrt{p} $ lub $ p/3 $, gdzie $ p $ to liczba wszystkich cech) jest wybierany w każdym węźle drzewa do określenia najlepszego podziału.

#### Krok 2: Budowa drzew
Każde drzewo $ h_t(x) $ jest trenowane na zbiorze $ D_t $. Drzewo podejmuje decyzje na podstawie reguł podziału opartych na wybranych cechach. W przeciwieństwie do pojedynczego drzewa decyzyjnego, las losowy ogranicza overfitting dzięki różnorodności drzew.

#### Krok 3: Agregacja predykcji
- Dla **klasyfikacji**:
  Predykcja lasu losowego to klasa, która otrzymała najwięcej głosów od drzew:
  $$
  \hat{y}(x) = \text{mode} \{ h_1(x), h_2(x), \dots, h_T(x) \},
  $$
  gdzie $ \text{mode} $ oznacza najczęściej występującą klasę.
- Dla **regresji**:
  Predykcja to średnia predykcji wszystkich drzew:
  $$
  \hat{y}(x) = \frac{1}{T} \sum_{t=1}^T h_t(x).
  $$

#### Krok 4: Ważność cech (Feature Importance)
Lasy losowe mogą oceniać ważność cech na podstawie tego, jak bardzo każda cecha przyczynia się do zmniejszenia nieczystości (np. entropii lub współczynnika Gini) w węzłach drzew. Ważność cechy $ j $ można zapisać jako:
$$
\text{Importance}(j) = \frac{1}{T} \sum_{t=1}^T \sum_{\text{węzły } v \text{ w drzewie } t} \Delta i(v, j),
$$
gdzie $ \Delta i(v, j) $ to zmniejszenie nieczystości w węźle $ v $ dzięki cesze $ j $.

### Zalety lasów losowych
- Odporność na overfitting dzięki agregacji wielu drzew.
- Dobra wydajność w zadaniach z dużą liczbą cech i szumem.
- Możliwość oceny ważności cech.
- Prosta implementacja i równoległe trenowanie drzew.

### Wady
- Wysoka złożoność obliczeniowa przy dużej liczbie drzew i danych.
- Mniejsza interpretowalność w porównaniu do pojedynczego drzewa decyzyjnego.
- Może być mniej skuteczny w zadaniach wymagających modelowania złożonych zależności (np. w porównaniu do gradient boosting).

---

### Zadanie nr 2 (3 punkty)
Zbuduj model lasów losowych w Pythonie, używając scikit-learn, i oceń jego dokładność na syntetycznym zbiorze danych klasyfikacji binarnej.

1. Wygeneruj zbiór danych binarnej klasyfikacji (użyj `make_classification`).
2. Podziel dane na zbiór treningowy i testowy.
3. Zbuduj model lasów losowych z 50 drzewami.
4. Oceń dokładność modelu na zbiorze testowym.

# Gradient Boosting

Gradient Boosting to metoda ensemble, która buduje model sekwencyjnie, dodając drzewa decyzyjne korygujące błędy poprzednich. Optymalizuje funkcję straty (np. log-loss dla klasyfikacji) za pomocą gradientów. 

### Kluczowe cechy:
- Sekwencyjne trenowanie drzew.
- Minimalizacja błędów poprzez gradient descent.
- Skuteczna, ale wymaga strojenia (np. learning rate).

### Matematyka:
W kroku $ t $, nowe drzewo $ h_t(x) $ minimalizuje ujemny gradient straty:
$$
h_t(x) \approx - \frac{\partial L(y, \hat{y}_{t-1})}{\partial \hat{y}_{t-1}}.
$$
Predykcja: 
$$
\hat{y}(x) = \sum_{t=1}^T \eta h_t(x),
$$
gdzie $ \eta $ to learning rate.

---

### Zadanie nr 3 (1 punkt)
Zbuduj model Gradient Boosting w Pythonie na danych `make_moons` i oceń jego dokładność.

### Instrukcje
1. Wygeneruj zbiór danych `make_moons` (500 próbek, szum 0.2).
2. Podziel dane na treningowe i testowe (30% testowe).
3. Zbuduj model Gradient Boosting z 100 drzewami i learning rate 0.1.
4. Wypisz dokładność na zbiorze testowym.

# Random Search

Random Search (ang. losowe przeszukiwanie) to metoda strojenia hiperparametrów modelu uczenia maszynowego, polegająca na losowym próbkowaniu kombinacji wartości hiperparametrów z zadanego zakresu, zamiast testowania wszystkich możliwych kombinacji (jak w Grid Search). Dla każdej losowej kombinacji model jest trenowany i oceniany (np. za pomocą walidacji krzyżowej), a najlepsza kombinacja jest wybierana.

### Kluczowe cechy:
- **Losowość**: Hiperparametry są losowane z zdefiniowanych rozkładów (np. jednorodnego, logarytmicznego).
- **Efektywność**: Testuje mniejszą liczbę kombinacji niż Grid Search, co przyspiesza proces dla dużych przestrzeni hiperparametrów.
- **Elastyczność**: Pozwala na określenie zakresów ciągłych (np. learning rate od 0.001 do 0.1) lub dyskretnych (np. liczba drzew: 50, 100, 200).

### Matematyka:
Niech $ \Theta $ to przestrzeń hiperparametrów (np. $\Theta = \{ \text{n\_estimators}, \text{learning\_rate}, \text{max\_depth} \}$). Random Search losuje $ n $ kombinacji $ \theta_1, \theta_2, \dots, \theta_n \in \Theta $ z rozkładu (np. jednorodnego) i ocenia model dla każdej $ \theta_i $ za pomocą funkcji straty $ L(\theta_i) $ (np. średni błąd walidacji krzyżowej). Wybiera:
$$
\theta^* = \arg\min_{\theta_i} L(\theta_i).
$$

### Dlaczego Random Search jest ważny?
- **Szybsze niż Grid Search**: W dużych przestrzeniach hiperparametrów testuje mniej kombinacji, zachowując wysoką szansę na znalezienie dobrych wartości.
- **Skuteczność w wysokich wymiarach**: Losowe próbkowanie lepiej eksploruje przestrzeń niż systematyczne grid search, zwłaszcza gdy tylko niektóre hiperparametry mają duże znaczenie.
- **Praktyczność**: Umożliwia strojenie modeli w rozsądnym czasie, co jest kluczowe w rzeczywistych zastosowaniach, gdzie zasoby obliczeniowe są ograniczone.

---


### Zadanie nr 4 (1 punkt)
Użyj Random Search do strojenia hiperparametrów modelu XGBoost na danych `make_moons`, testując większą liczbę hiperparametrów, i wypisz najlepszą dokładność oraz wybrane hiperparametry.

### Instrukcje
1. Wygeneruj zbiór danych `make_moons` (500 próbek, szum 0.2).
2. Podziel dane na treningowe i testowe (30% testowe).
3. Użyj `RandomizedSearchCV`, aby znaleźć najlepsze hiperparametry dla XGBoost, testując: `n_estimators`, `learning_rate`, `max_depth`, `min_child_weight`, `subsample`.
4. Wypisz najlepszą dokładność i najlepsze hiperparametry.